### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('one_hot_encoder_geography.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [3]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model



In [4]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [5]:

# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [6]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


/Users/sriramsohan/Documents/projects/testing_models/pipeline_inf_env/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
2024-12-25 17:15:08.414843: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-12-25 17:15:08.414868: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-12-25 17:15:08.414874: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-12-25 17:15:08.414902: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-25 17:15:08.414916: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/t

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50


2024-12-25 17:15:09.729631: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-12-25 17:15:09.773754: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-12-25 17:15:09.783011: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-12-25 17:15:09.819014: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-12-25 17:15:09.827971: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-12-25 17:15:09.988385: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-12-25 17:15:10.014178: I tensorflow/core/grappler/optimizers/cust

167/167 [==============================] - 6s 15ms/step - loss: 0.6521 - accuracy: 0.6542
Epoch 2/50
Epoch 2/50
167/167 [==============================] - 6s 15ms/step - loss: 0.5616 - accuracy: 0.7186
Epoch 2/50
167/167 [==============================] - 6s 15ms/step - loss: 0.6873 - accuracy: 0.6301
Epoch 2/50
167/167 [==============================] - 6s 15ms/step - loss: 0.5838 - accuracy: 0.7232
Epoch 2/50
167/167 [==============================] - 6s 15ms/step - loss: 0.5926 - accuracy: 0.6925
Epoch 2/50
167/167 [==============================] - 6s 15ms/step - loss: 0.5109 - accuracy: 0.7606
Epoch 2/50
167/167 [==============================] - 6s 15ms/step - loss: 0.5209 - accuracy: 0.7555
Epoch 2/50
167/167 [==============================] - 6s 17ms/step - loss: 0.5175 - accuracy: 0.7579
Epoch 2/50
167/167 [==============================] - 2s 10ms/step - loss: 0.4630 - accuracy: 0.8104
Epoch 3/50
167/167 [==============================] - 2s 10ms/step - loss: 0.4432 - accurac

...

167/167 [==============================] - 2s 10ms/step - loss: 1.5017 - accuracy: 0.7515
Epoch 74/100
167/167 [==============================] - 2s 10ms/step - loss: 3.4398 - accuracy: 0.7307
Epoch 80/100
Epoch 49/100
167/167 [==============================] - 2s 10ms/step - loss: 1.1981 - accuracy: 0.7390
Epoch 99/100
167/167 [==============================] - 2s 9ms/step - loss: 2.3082 - accuracy: 0.7386
Epoch 75/100
167/167 [==============================] - 1s 9ms/step - loss: 0.5280 - accuracy: 0.7958
Epoch 6/100
167/167 [==============================] - 1s 8ms/step - loss: 1.2977 - accuracy: 0.7330
Epoch 52/100
  1/167 [..............................] - ETA: 1s - loss: 0.6061 - accuracy: 0.8438Epoch 50/100
Epoch 81/100
167/167 [==============================] - 1s 8ms/step - loss: 1.2766 - accuracy: 0.7462
Epoch 100/100
167/167 [==============================] - 2s 9ms/step - loss: 0.4741 - accuracy: 0.7810
Epoch 2/100
167/167 [==============================] - 1s 8ms/step - lo

/Users/sriramsohan/Documents/projects/testing_models/pipeline_inf_env/lib/python3.10/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
2024-12-25 17:25:39.348876: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-12-25 17:25:39.348915: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-12-25 17:25:39.348929: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-12-25 17:25:39.348956: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-25 17:25:39.348969: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/t

Epoch 1/50


2024-12-25 17:25:40.075526: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 2s 3ms/step - loss: 0.5230 - accuracy: 0.7538
Epoch 2/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4355 - accuracy: 0.8086
Epoch 3/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4336 - accuracy: 0.8109
Epoch 4/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4337 - accuracy: 0.8106
Epoch 5/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4337 - accuracy: 0.8085
Epoch 6/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4337 - accuracy: 0.8124
Epoch 7/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4339 - accuracy: 0.8109
Epoch 8/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4332 - accuracy: 0.8099
Epoch 9/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4342 - accuracy: 0.8114
Epoch 10/50
250/250 [==============================] - 1s 3ms/step - loss: 0.4344 - accuracy: 0.8095
Epoch 11/5